<article class="day-desc"><h2>--- Day 12: Passage Pathing ---</h2><p>With your <span title="Sublime.">submarine's subterranean subsystems subsisting suboptimally</span>, the only way you're getting out of this cave anytime soon is by finding a path yourself. Not just <em>a</em> path - the only way to know if you've found the <em>best</em> path is to find <em>all</em> of them.</p>
<p>Fortunately, the sensors are still mostly working, and so you build a rough map of the remaining caves (your puzzle input). For example:</p>
<pre><code>start-A
start-b
A-c
A-b
b-d
A-end
b-end
</code></pre>
<p>This is a list of how all of the caves are connected. You start in the cave named <code>start</code>, and your destination is the cave named <code>end</code>. An entry like <code>b-d</code> means that cave <code>b</code> is connected to cave <code>d</code> - that is, you can move between them.</p>
<p>So, the above cave system looks roughly like this:</p>
<pre><code>    start
    /   \
c--A-----b--d
    \   /
     end
</code></pre>
<p>Your goal is to find the number of distinct <em>paths</em> that start at <code>start</code>, end at <code>end</code>, and don't visit small caves more than once. There are two types of caves: <em>big</em> caves (written in uppercase, like <code>A</code>) and <em>small</em> caves (written in lowercase, like <code>b</code>). It would be a waste of time to visit any small cave more than once, but big caves are large enough that it might be worth visiting them multiple times. So, all paths you find should <em>visit small caves at most once</em>, and can <em>visit big caves any number of times</em>.</p>
<p>Given these rules, there are <code><em>10</em></code> paths through this example cave system:</p>
<pre><code>start,A,b,A,c,A,end
start,A,b,A,end
start,A,b,end
start,A,c,A,b,A,end
start,A,c,A,b,end
start,A,c,A,end
start,A,end
start,b,A,c,A,end
start,b,A,end
start,b,end
</code></pre>
<p>(Each line in the above list corresponds to a single path; the caves visited by that path are listed in the order they are visited and separated by commas.)</p>
<p>Note that in this cave system, cave <code>d</code> is never visited by any path: to do so, cave <code>b</code> would need to be visited twice (once on the way to cave <code>d</code> and a second time when returning from cave <code>d</code>), and since cave <code>b</code> is small, this is not allowed.</p>
<p>Here is a slightly larger example:</p>
<pre><code>dc-end
HN-start
start-kj
dc-start
dc-HN
LN-dc
HN-end
kj-sa
kj-HN
kj-dc
</code></pre>
<p>The <code>19</code> paths through it are as follows:</p>
<pre><code>start,HN,dc,HN,end
start,HN,dc,HN,kj,HN,end
start,HN,dc,end
start,HN,dc,kj,HN,end
start,HN,end
start,HN,kj,HN,dc,HN,end
start,HN,kj,HN,dc,end
start,HN,kj,HN,end
start,HN,kj,dc,HN,end
start,HN,kj,dc,end
start,dc,HN,end
start,dc,HN,kj,HN,end
start,dc,end
start,dc,kj,HN,end
start,kj,HN,dc,HN,end
start,kj,HN,dc,end
start,kj,HN,end
start,kj,dc,HN,end
start,kj,dc,end
</code></pre>
<p>Finally, this even larger example has <code>226</code> paths through it:</p>
<pre><code>fs-end
he-DX
fs-he
start-DX
pj-DX
end-zg
zg-sl
zg-pj
pj-he
RW-he
fs-DX
pj-RW
zg-RW
start-pj
he-WI
zg-he
pj-fs
start-RW
</code></pre>
<p><em>How many paths through this cave system are there that visit small caves at most once?</em></p>
</article>

In [1]:
function init()
    adjacency_map = Dict{String, Set{String}}();
    for line in readlines("input.txt")
        (cave0, cave1) = split(line, "-");
        if !haskey(adjacency_map, cave0)
            adjacency_map[cave0] = Set{String}([]);
        end;
        push!(adjacency_map[cave0], cave1);
        if !haskey(adjacency_map, cave1)
            adjacency_map[cave1] = Set{String}([]);
        end;
        push!(adjacency_map[cave1], cave0);
    end;
    return adjacency_map;
end;

In [2]:
function is_upper(s::String)
    for c in s
        if !isuppercase(c)
            return false;
        end;
    end;
    return true;
end;

In [3]:
mutable struct path_node{T}
    val::T
    parent::Union{path_node{T}, Nothing}
end;

function in_path(node::Nothing, val)::Bool
    return false;
end;

function in_path(node::path_node, val)::Bool
    return (node.val == val) || in_path(node.parent, val);
end;

function print_path(node::path_node)
    path = Vector{String}([]);
    current = node;
    while !isnothing(current)
        prepend!(path, [current.val]);
        current = current.parent;
    end;
    println(join(path, ","));
end;

In [4]:
adjacency_map = init();
start = path_node{String}("start", nothing);
non_terminal = Vector{path_node{String}}([start]);
terminal = Vector{path_node{String}}([]);
while length(non_terminal) != 0
    current_node = pop!(non_terminal);
    for next in adjacency_map[current_node.val]
        if next == "end"
            push!(terminal, path_node{String}(next, current_node));
        elseif is_upper(next)
            push!(non_terminal, path_node{String}(next, current_node));
        elseif !in_path(current_node, next)
            push!(non_terminal, path_node{String}(next, current_node))
        end;  
    end;
end;

In [5]:
length(terminal)

4912

<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>After reviewing the available paths, you realize you might have time to visit a single small cave <em>twice</em>. Specifically, big caves can be visited any number of times, a single small cave can be visited at most twice, and the remaining small caves can be visited at most once. However, the caves named <code>start</code> and <code>end</code> can only be visited <em>exactly once each</em>: once you leave the <code>start</code> cave, you may not return to it, and once you reach the <code>end</code> cave, the path must end immediately.</p>
<p>Now, the <code>36</code> possible paths through the first example above are:</p>
<pre><code>start,A,b,A,b,A,c,A,end
start,A,b,A,b,A,end
start,A,b,A,b,end
start,A,b,A,c,A,b,A,end
start,A,b,A,c,A,b,end
start,A,b,A,c,A,c,A,end
start,A,b,A,c,A,end
start,A,b,A,end
start,A,b,d,b,A,c,A,end
start,A,b,d,b,A,end
start,A,b,d,b,end
start,A,b,end
start,A,c,A,b,A,b,A,end
start,A,c,A,b,A,b,end
start,A,c,A,b,A,c,A,end
start,A,c,A,b,A,end
start,A,c,A,b,d,b,A,end
start,A,c,A,b,d,b,end
start,A,c,A,b,end
start,A,c,A,c,A,b,A,end
start,A,c,A,c,A,b,end
start,A,c,A,c,A,end
start,A,c,A,end
start,A,end
start,b,A,b,A,c,A,end
start,b,A,b,A,end
start,b,A,b,end
start,b,A,c,A,b,A,end
start,b,A,c,A,b,end
start,b,A,c,A,c,A,end
start,b,A,c,A,end
start,b,A,end
start,b,d,b,A,c,A,end
start,b,d,b,A,end
start,b,d,b,end
start,b,end
</code></pre>
<p>The slightly larger example above now has <code>103</code> paths through it, and the even larger example now has <code>3509</code> paths through it.</p>
<p>Given these new rules, <em>how many paths through this cave system are there?</em></p>
</article>

In [6]:
mutable struct path_node_2{T}
    val::T
    parent::Union{path_node_2{T}, Nothing}
    has_revisited::Bool
end;

function in_path(node::Nothing, val)::Bool
    return false;
end;

function in_path(node::path_node_2, val)::Bool
    return (node.val == val) || in_path(node.parent, val);
end;

function print_path(node::path_node_2)
    path = Vector{String}([]);
    current = node;
    while !isnothing(current)
        prepend!(path, [current.val]);
        current = current.parent;
    end;
    println(join(path, ","));
end;

In [10]:
adjacency_map = init();
start = path_node_2{String}("start", nothing, false);
non_terminal = Vector{path_node_2{String}}([start]);
terminal = Vector{path_node_2{String}}([]);
while length(non_terminal) != 0
    current_node = pop!(non_terminal);
    for next in adjacency_map[current_node.val]
        if next == "start"
            nothing;
        elseif next == "end"
            push!(terminal, path_node_2{String}(next, current_node, current_node.has_revisited));
        elseif is_upper(next)
            push!(non_terminal, path_node_2{String}(next, current_node, current_node.has_revisited));
        elseif !in_path(current_node, next)
            push!(non_terminal, path_node_2{String}(next, current_node, current_node.has_revisited))
        elseif !current_node.has_revisited
            push!(non_terminal, path_node_2{String}(next, current_node, true));
        end;
    end;
end;

In [ ]:
termi